In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import random
import time
import sklearn.metrics as metrics
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
import numpy as np
import heapq
from tqdm import tqdm

class KNearestNeighbor:
    def __init__(self):
        self.X_train = None
        self.y_train = None
        self.k = None
        self.valuep = None


    # Distance calculation
    def cal_dist(self, p, q):  # p, q are two rows in the data matrix
        return np.sum(np.abs(p - q) ** self.valuep) ** (1 / self.valuep)
    

    def fit(self, X, y, k, valuep=2):
        self.X_train = X
        self.y_train = y
        self.k = k
        self.valuep = valuep

    
    # def predict(self, X_test):
    #     X_train = self.X_train
    #     y_train = self.y_train
    #     y_pred = []
    #     # if issparse(X_train):
    #     #     X_train = X_train.toarray()
    #     #     y_train = y_train.toarray()
    #     for i in range(0, X_test.shape[0]):
    #         distance = []
    #         for j in range(0, X_train.shape[0]):
    #             distance.append((j, self.cal_dist(X_test[i], X_train[j])))  # A tuple which stores index and distance
    #         distance = sorted(distance, key=lambda x: x[1])
    #         neighbors = distance[0:self.k]
    #         # print(neighbors)
    #         # print([y_train[x[0]] for x in neighbors])
    #         majority = self.find_majority([y_train[x[0]] for x in neighbors])
    #         y_pred.append(majority)
    #     return y_pred
    

    def predict(self, X_test):
        y_pred = []
        
        # dist_matrix = np.zeros((X_test.shape[0], X_train.shape[0]))
        for i in tqdm(range(X_test.shape[0])):
            neighbors = []
            for j in range(self.X_train.shape[0]):
                dist = self.cal_dist(X_test[i], self.X_train[j])
                self.neighboring(neighbors, dist, self.y_train[j])
            class_count = {}
            for _, c in neighbors:
                class_count[c] = class_count.get(c, 0) + 1
            majority = max(class_count, key=class_count.get)
            y_pred.append(majority)
        return y_pred


    def neighboring(self, neighbors, dist, y_class):
        if len(neighbors) < self.k:
            heapq.heappush(neighbors, (-dist, y_class))
        else:
            heapq.heappushpop(neighbors, (-dist, y_class))

In [4]:
mydata = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", header=None)
mydata.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 
              'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'target']
mydata

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [5]:
testdata = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test", header=None, skiprows=1)
testdata.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 
                    'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'target']
testdata['target'] = testdata['target'].str.replace('.', '', regex=False)
testdata

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
16277,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K
16278,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
16279,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


In [6]:
mydata.replace({' ?': np.NaN}, inplace=True)
print(mydata.isnull().sum())
mydata.dropna(inplace=True)

age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
target               0
dtype: int64


In [7]:
testdata.replace({' ?': np.NaN}, inplace=True)
print(testdata.isnull().sum())
testdata.dropna(inplace=True)

age                 0
workclass         963
fnlwgt              0
education           0
education-num       0
marital-status      0
occupation        966
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country    274
target              0
dtype: int64


In [8]:
X_train = mydata.drop(['target'], axis=1)
y_train = mydata.target

X_test = testdata.drop(['target'], axis=1)
y_test = testdata.target

In [9]:
numeric_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
numeric_transformer = preprocessing.Normalizer()

categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
# categorical_transformer = preprocessing.OneHotEncoder(drop='first')
categorical_transformer = preprocessing.OrdinalEncoder()

preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),
                                               ('cat', categorical_transformer, categorical_features)])

In [10]:
X_train = preprocessor.fit_transform(X_train)
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)

X_test = preprocessor.transform(X_test)
y_test = le.transform(y_test)

In [11]:
# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', KNeighborsClassifier)])

In [20]:
# find the best k
random.seed(10)
rndix = random.sample(range(0, X_test.shape[0]), 500)
samplex = X_test[rndix, :]
sampley = y_test[rndix]
for k in range(1, 11):
    clf = KNearestNeighbor()
    clf.fit(X_train, y_train, k=k, valuep=2)
    sample_pred = clf.predict(samplex)
    print(accuracy_score(sampley, sample_pred))

100%|██████████| 500/500 [02:22<00:00,  3.52it/s]


0.784


100%|██████████| 500/500 [02:21<00:00,  3.54it/s]


0.778


100%|██████████| 500/500 [02:21<00:00,  3.54it/s]


...

0.81


100%|██████████| 500/500 [02:21<00:00,  3.54it/s]


0.814


100%|██████████| 500/500 [02:22<00:00,  3.52it/s]

0.812


In [13]:
st = time.time()

clf = KNearestNeighbor()
clf.fit(X_train, y_train, k=5, valuep=2)
y_pred = clf.predict(X_test)

100%|██████████| 15060/15060 [1:10:23<00:00,  3.57it/s]


In [14]:
accuracy_score(y_test, y_pred)

0.8148738379814077

In [15]:
et = time.time()
print('Execution time: ', et - st, ' seconds')

Execution time:  4228.911317586899  seconds


In [18]:
confusion_matrix(y_test, y_pred)

array([[10245,  1115],
       [ 1673,  2027]])

In [19]:
classification_report(y_test, y_pred,labels=np.unique(y_pred))

'              precision    recall  f1-score   support\n\n           0       0.86      0.90      0.88     11360\n           1       0.65      0.55      0.59      3700\n\n    accuracy                           0.81     15060\n   macro avg       0.75      0.72      0.74     15060\nweighted avg       0.81      0.81      0.81     15060\n'

In [16]:
clf = KNeighborsClassifier(n_neighbors=5, p=2)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [17]:
accuracy_score(y_test, y_pred)

0.8148738379814077